[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NOAA-EPIC/global-eagle/blob/feature/hello_world/examples/getting_started/colab_notebook_demo/pipeline_demo.ipynb)

# Welcome!

#### Install necessary libraries

Runtime: 3 minutes

In [ ]:
!pip install anemoi-datasets==0.5.25 anemoi-graphs==0.6.2 anemoi-models==0.8.1 anemoi-training==0.5.1 anemoi-inference==0.6.3 trimesh 'numpy<2.3' 'earthkit-data<0.14.0' ufs2arco

#### Clone repository:

In [ ]:
!git clone -b feature/hello_world https://github.com/NOAA-EPIC/global-eagle.git

#TODO -- right before merging to main we need to update this to not load branch.

#### Create training and validation datasets with ufs2arco

In [ ]:
!ufs2arco global-eagle/examples/getting_started/colab_notebook_demo/data/replay.yaml

#### Train a model with anemoi-core modules

In [ ]:
import os
os.environ["ANEMOI_BASE_SEED"] = "42"
os.environ["SLURM_JOB_ID"] = "0"

In [ ]:
%cd global-eagle/examples/getting_started/colab_notebook_demo/train/

In [ ]:
!anemoi-training train --config-name=config

#### Create a forecast with anemoi-inference

In [ ]:
%cd /content/global-eagle/examples/getting_started/colab_notebook_demo/inference/

In [ ]:
!anemoi-inference run inference_config.yaml

View inference

In [ ]:
import xarray as xr
ds = xr.open_dataset("2022-01-03T00.nc")
ds

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fhr = 1
temp = ds['tmp2m'].isel(time=fhr).values
lat = ds['latitude'].values
lon = ds['longitude'].values

# Plotplt.figure(figsize=(10, 6))
plt.scatter(lon, lat, c=temp, s=10, cmap='coolwarm')
plt.colorbar(label='2m Temperature')
plt.title(f'2m Temperature at {ds["time"][fhr].values}')
plt.show()

Postprocess inference

In [ ]:
!python postprocess.py

In [ ]:
ds_post = xr.open_dataset("2022-01-03T00_postprocessed.nc")
ds_post

#### Verify the forecast against GFS with wxvx

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() 

Go to your terminal and run the following commands:

`conda install -y -c ufs-community -c paul.madden wxvx`

`conda activate wxvx`

`cd global-eagle/examples/getting_started/colab_notebook_demo/verification/`

`wxvx -c wxvx_config.yaml -t plots`